# Memory Usage Measurement

In [ ]:
from benchmark.environment import start, remove, IMPROVEMENTS, STATES
from benchmark.api import (
    ApiBeforeEnvironmentIndependence,
    ApiAfterEnvironmentIndependence,
    ApiServiceMerge,
)
from time import sleep
from os import chdir
from pathlib import Path
from itertools import product
from tqdm.auto import tqdm, trange
import pandas as pd
import docker

In [ ]:
if not "PROJECT_DIR" in globals():
    PROJECT_DIR = Path().resolve().parent
    chdir(PROJECT_DIR)

if not "DATA_DIR" in globals():
    DATA_DIR = PROJECT_DIR / "memory-usage" / "data"
    DATA_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
states = list(product(IMPROVEMENTS, STATES))
states

In [ ]:
docker_client = docker.from_env()

To measure the memory usage of the deployed Docker containers of DBrepo, we will use the `docker stats` command.

In [ ]:
def measure_memory_usage() -> pd.DataFrame:
    return pd.DataFrame(
        [
            {
                "service": service.name,
                "memory-usage": service.stats(stream=False)["memory_stats"]["usage"],
            }
            for service in docker_client.containers.list()
        ]
    )

We first make sure that any previous deployment is removed.

In [ ]:
for improvement, state in tqdm(states):
    remove(improvement, state)

After that we deploy each state of DBRepo respectively and measure the memory usage of its Docker container corresponding to its services.

In [ ]:
for improvement, state in tqdm(states):
    start(improvement, state)

    sleep(1)

    if improvement == "environment-independence":
        if state == "before":
            api = ApiBeforeEnvironmentIndependence()
        else:
            api = ApiAfterEnvironmentIndependence()
    else:
        api = ApiServiceMerge()

    api.create_user("albert", "einstein", "albert.einstein@example.com")
    api.login("albert", "einstein")

    memory_usage = measure_memory_usage()
    memory_usage["databases"] = 0

    database_memory_usage = [memory_usage]

    for databases in trange(1, 11, leave=False):
        api.create_database(f"Database {databases}")

        memory_usage = measure_memory_usage()
        memory_usage["databases"] = databases
        database_memory_usage.append(memory_usage)

    database_memory_usage = pd.concat(database_memory_usage)
    database_memory_usage.to_csv(DATA_DIR / f"{improvement}-{state}.csv", index=False)

    remove(improvement, state)